# Research Topic: Water Quality Monitoring System using Recurrent Neural Network (RNNs) and Internet of Things (IoT)


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score


In [5]:
data = pd.read_csv("C:/Users/muham/OneDrive/Desktop/python_folder/Water_collection/brisbane_water_quality/brisbane_water_quality.csv")

In [6]:
data.head()

,Timestamp,Record number,Average Water Speed,Average Water Direction,Chlorophyll,Chlorophyll [quality],Temperature,Temperature [quality],Dissolved Oxygen,Dissolved Oxygen [quality],Dissolved Oxygen (%Saturation),Dissolved Oxygen (%Saturation) [quality],pH,pH [quality],Salinity,Salinity [quality],Specific Conductance,Specific Conductance [quality],Turbidity,Turbidity [quality]
0,2023-08-04 23:00:00,1468,4.834,73.484,1.621,NaN,20.018,NaN,7.472,NaN,101.175,NaN,8.176,NaN,35.215,NaN,53.262,NaN,2.068,NaN
1,2023-08-04 23:30:00,1469,2.544,106.424,1.959,NaN,19.986,NaN,7.455,NaN,100.884,NaN,8.175,NaN,35.209,NaN,53.254,NaN,1.994,NaN
2,2023-08-04 23:00:00,1470,1.260,156.755,1.620,NaN,20.001,NaN,7.430,NaN,100.571,NaN,8.171,NaN,35.207,NaN,53.252,NaN,2.030,NaN
3,2023-08-04 23:30:00,1471,0.760,281.754,1.761,NaN,19.983,NaN,7.419,NaN,100.398,NaN,8.171,NaN,35.211,NaN,53.257,NaN,1.973,NaN
4,2023-08-04 23:00:00,1472,3.397,244.637,1.635,NaN,19.986,NaN,7.429,NaN,100.538,NaN,8.171,NaN,35.208,NaN,53.253,NaN,1.944,NaN


In [7]:
# Identify non-numeric columns
non_numeric_columns = data.select_dtypes(exclude=[np.number]).columns
print(non_numeric_columns)

# Fill NaNs only in numeric columns
numeric_columns = data.select_dtypes(include=[np.number]).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())


Index(['Timestamp'], dtype='object')


In [8]:
# Drop rows with NaNs in the remaining columns
data_clean = data.dropna(subset=['Average Water Speed', 'Average Water Direction', 'Chlorophyll', 
                                 'Chlorophyll [quality]', 'Temperature', 'Temperature [quality]',
                                 'Dissolved Oxygen', 'Dissolved Oxygen [quality]', 
                                 'Dissolved Oxygen (%Saturation)', 'pH', 'Salinity', 
                                 'Specific Conductance', 'Turbidity'])

# Select relevant features and target
features = ['Average Water Speed', 'Average Water Direction', 'Chlorophyll', 
            'Chlorophyll [quality]', 'Temperature [quality]',
            'Dissolved Oxygen', 'Dissolved Oxygen [quality]', 
            'Dissolved Oxygen (%Saturation)', 'pH', 'Salinity', 
            'Specific Conductance', 'Turbidity']
target = 'Temperature'

# Prepare the feature and target arrays
X = data_clean[features].values
y = data_clean[target].values


In [9]:
# Normalize data
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()


In [10]:
# Check if normalization worked
print(np.any(np.isnan(X_scaled)))
print(np.any(np.isinf(X_scaled)))
print(np.any(np.isnan(y_scaled)))
print(np.any(np.isinf(y_scaled)))


False
False
False
False


In [11]:
# Create sequences
def create_sequences(X, y, time_steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i + time_steps])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)


In [12]:
time_steps = 10
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)


In [13]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4, clipvalue=1.0), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


c:\Users\muham\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 29s 18ms/step - loss: 0.0723 - val_loss: 0.0294
Epoch 2/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - loss: 0.0286 - val_loss: 0.0181
Epoch 3/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 0.0210 - val_loss: 0.0164
Epoch 4/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0194 - val_loss: 0.0162
Epoch 5/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 0.0189 - val_loss: 0.0155
Epoch 6/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 0.0177 - val_loss: 0.0156
Epoch 7/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 0.0176 - val_loss: 0.0146
Epoch 8/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 0.0167 - val_loss: 0.0143
Epoch 9/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 0.0162 - val_loss: 0.0140
Epoch 10/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0161 - val_loss: 0.0149
Epoch 11/50
773/773 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 0.0155 - val_loss: 0.0162
Epoch 12/50
773/773 ━━━━━━━━━━

In [14]:

# Make predictions
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate metrics
mse = mean_squared_error(y_test_original, y_pred)
r2 = r2_score(y_test_original, y_pred)

print('Mean Squared Error:', mse)
print('R-Squared:', r2)


194/194 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
Mean Squared Error: 1.5743688965628602
R-Squared: 0.8420559043627323
